In [1]:
# secure_vpn_demo.py
# A custom Python-based secure VPN-like server for educational purposes

import socket
import threading
import hashlib
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64
import getpass

# Configuration
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5555
BUFFER_SIZE = 4096
AES_KEY_SIZE = 32  # 256-bit key
AES_IV_SIZE = 16   # 128-bit IV

# Simulated user database (in a real app, use proper secure storage)
users = {
    'user1': hashlib.sha256('password1'.encode()).hexdigest(),
    'user2': hashlib.sha256('password2'.encode()).hexdigest()
}

class SecureVPN:
    def __init__(self):
        self.sessions = {}
        
    def generate_key_iv(self):
        """Generate random key and IV for AES encryption"""
        key = os.urandom(AES_KEY_SIZE)
        iv = os.urandom(AES_IV_SIZE)
        return key, iv
    
    def encrypt(self, data, key, iv):
        """Encrypt data with AES-CBC"""
        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(data, AES.block_size)
        return cipher.encrypt(padded_data)
    
    def decrypt(self, encrypted_data, key, iv):
        """Decrypt data with AES-CBC"""
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted_data = cipher.decrypt(encrypted_data)
        return unpad(decrypted_data, AES.block_size)
    
    def start_server(self):
        """Start the VPN server"""
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((SERVER_HOST, SERVER_PORT))
        server_socket.listen(5)
        print(f"[*] VPN Server listening on {SERVER_HOST}:{SERVER_PORT}")
        
        while True:
            client_socket, client_address = server_socket.accept()
            print(f"[+] New connection from {client_address}")
            
            # Handle client in a new thread
            client_thread = threading.Thread(
                target=self.handle_client,
                args=(client_socket,)
            )
            client_thread.start()
    
    def authenticate_client(self, client_socket):
        """Handle client authentication"""
        try:
            # Receive username
            username = client_socket.recv(BUFFER_SIZE).decode().strip()
            if not username or username not in users:
                client_socket.send(b'AUTH_FAIL')
                return None
            
            # Request password
            client_socket.send(b'PASSWORD')
            
            # Receive password hash
            password_hash = client_socket.recv(BUFFER_SIZE).decode().strip()
            
            # Verify credentials
            if users[username] == password_hash:
                client_socket.send(b'AUTH_SUCCESS')
                print(f"[*] Client {username} authenticated successfully")
                return username
            else:
                client_socket.send(b'AUTH_FAIL')
                return None
        except Exception as e:
            print(f"[-] Authentication error: {e}")
            return None
    
    def handle_client(self, client_socket):
        """Handle client connection"""
        try:
            # Step 1: Authentication
            username = self.authenticate_client(client_socket)
            if not username:
                client_socket.close()
                return
            
            # Step 2: Key exchange (in a real VPN, use proper key exchange like DH)
            key, iv = self.generate_key_iv()
            
            # Send encrypted key/IV (in real app, use asymmetric encryption here)
            client_socket.send(base64.b64encode(key))
            ack = client_socket.recv(BUFFER_SIZE)
            client_socket.send(base64.b64encode(iv))
            
            # Store session
            self.sessions[username] = {'key': key, 'iv': iv, 'socket': client_socket}
            print(f"[*] Established secure channel with {username}")
            
            # Step 3: Handle secure communication
            while True:
                encrypted_data = client_socket.recv(BUFFER_SIZE)
                if not encrypted_data:
                    break
                
                # Decrypt the data
                try:
                    decrypted_data = self.decrypt(encrypted_data, key, iv)
                    print(f"[{username}] Received: {decrypted_data.decode()}")
                    
                    # Echo back (in a real VPN, this would route traffic)
                    response = f"Server received: {decrypted_data.decode()}"
                    encrypted_response = self.encrypt(response.encode(), key, iv)
                    client_socket.send(encrypted_response)
                except Exception as e:
                    print(f"[-] Decryption error: {e}")
                    break
            
        except Exception as e:
            print(f"[-] Client handling error: {e}")
        finally:
            if username in self.sessions:
                del self.sessions[username]
            client_socket.close()
            print(f"[-] Connection closed for {username}")

class VPNClient:
    def __init__(self, server_host, server_port):
        self.server_host = server_host
        self.server_port = server_port
        self.key = None
        self.iv = None
        self.socket = None
    
    def connect(self):
        """Connect to the VPN server"""
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.server_host, self.server_port))
        
        # Authentication
        username = input("Username: ")
        self.socket.send(username.encode())
        
        response = self.socket.recv(BUFFER_SIZE)
        if response != b'PASSWORD':
            print("Authentication failed")
            return False
        
        password = getpass.getpass("Password: ")
        password_hash = hashlib.sha256(password.encode()).hexdigest()
        self.socket.send(password_hash.encode())
        
        auth_result = self.socket.recv(BUFFER_SIZE)
        if auth_result != b'AUTH_SUCCESS':
            print("Authentication failed")
            return False
        
        # Key exchange
        self.key = base64.b64decode(self.socket.recv(BUFFER_SIZE))
        self.socket.send(b'ACK')
        self.iv = base64.b64decode(self.socket.recv(BUFFER_SIZE))
        
        print("[*] Connected to VPN server")
        return True
    
    def encrypt(self, data):
        """Encrypt data with AES-CBC"""
        cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        padded_data = pad(data, AES.block_size)
        return cipher.encrypt(padded_data)
    
    def decrypt(self, encrypted_data):
        """Decrypt data with AES-CBC"""
        cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        decrypted_data = cipher.decrypt(encrypted_data)
        return unpad(decrypted_data, AES.block_size)
    
    def send_message(self, message):
        """Send encrypted message to server"""
        encrypted_msg = self.encrypt(message.encode())
        self.socket.send(encrypted_msg)
        
        # Wait for response
        encrypted_response = self.socket.recv(BUFFER_SIZE)
        decrypted_response = self.decrypt(encrypted_response)
        print(f"[Server] {decrypted_response.decode()}")
    
    def close(self):
        """Close the connection"""
        if self.socket:
            self.socket.close()

def start_vpn_server():
    """Start the VPN server in foreground"""
    vpn = SecureVPN()
    vpn.start_server()

def start_vpn_client():
    """Start the VPN client"""
    client = VPNClient(SERVER_HOST, SERVER_PORT)
    if client.connect():
        try:
            while True:
                message = input("Message (or 'exit' to quit): ")
                if message.lower() == 'exit':
                    break
                client.send_message(message)
        finally:
            client.close()
    print("Client session ended")

def main():
    print("Secure VPN-like Communication Demo")
    print("==================================")
    print("1. Start VPN Server")
    print("2. Start VPN Client")
    print("3. Exit")
    
    choice = input("Select option (1-3): ")
    
    if choice == '1':
        print("Starting VPN server...")
        start_vpn_server()
    elif choice == '2':
        print("Starting VPN client...")
        start_vpn_client()
    elif choice == '3':
        print("Exiting...")
    else:
        print("Invalid choice")

if __name__ == "__main__":
    main()

<class 'ModuleNotFoundError'>: No module named 'Crypto'